In [1]:
import torch

import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from torch.optim import Adam

In [2]:
royal = ['queen', 'king', 'prince', 'princess']
plain = ['woman', 'man', 'girl', 'boy']
peafowl = ['peacock', 'peahen', 'peachick']
adult = ['queen', 'king', 'woman', 'man', 'peacock', 'peahen']
female = ['queen', 'princess', 'woman', 'girl', 'peahen']
male = ['king', 'prince', 'man', 'boy', 'peacock']
young = ['prince', 'princess', 'girl', 'boy', 'peachick']

union_data = [royal, plain, peafowl, adult, female, male, young]

In [3]:
all_data = royal + plain + peafowl + adult + female + male + young
vocabulary_size = 0

data_dict = {}

for data in all_data:
    if data not in data_dict:
        data_dict[data] = vocabulary_size
        vocabulary_size += 1

one_hot = []

for i in range(vocabulary_size):
    arr = [0] * vocabulary_size
    arr[i] = 1
    one_hot.append(arr)

print(data_dict)
print('vocabulary_size', vocabulary_size)

{'queen': 0, 'king': 1, 'prince': 2, 'princess': 3, 'woman': 4, 'man': 5, 'girl': 6, 'boy': 7, 'peacock': 8, 'peahen': 9, 'peachick': 10}
vocabulary_size 11


In [4]:
mat = []

for i in range(vocabulary_size):
    mat.append([0] * vocabulary_size)

for data in union_data:
    for value1 in data:
        for value2 in data:
            mat[data_dict[value1]][data_dict[value2]] += 1

for arr in mat:
    arr_sum = sum(arr)
    for i in range(vocabulary_size):
        arr[i] /= arr_sum

print(mat)

[[0.2, 0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.0, 0.06666666666666667, 0.13333333333333333, 0.0], [0.13333333333333333, 0.2, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.0, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.0], [0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.14285714285714285, 0.0, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.0, 0.07142857142857142], [0.14285714285714285, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.07142857142857142, 0.0, 0.14285714285714285, 0.07142857142857142, 0.0, 0.07142857142857142, 0.07142857142857142], [0.13333333333333333, 0.06666666666666667, 0.0, 0.06666666666666667, 0.2, 0.13333333333333333, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.0], [0.06666666666666667, 0.13333333333333333,

In [38]:
class WordEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Linear(11, 3)
        self.decoder = nn.Linear(3, 11)

    def forward(self, x):
        x = self.encoder(x)
        x = F.relu(x)
        x = self.decoder(x)
        x = F.softmax(x)

        return x

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
# model
model = WordEmbedding().to(device)

# loss
loss_fn = nn.MSELoss()

# Optimizer
optimizer = Adam(model.parameters())

inputs = torch.FloatTensor(one_hot).to(device)
labels = torch.tensor(mat).to(device)

for i in range(10000):
    # Zero your gradients for every batch!
    optimizer.zero_grad()

    outputs = model(inputs)
    
    loss = loss_fn(outputs, labels)
    loss.backward()

    optimizer.step()
    
    if i % 200 == 0:
        print(i)
        print(loss.item())
        print('---------')

C:\Users\Dima\AppData\Local\Temp\ipykernel_1796\1100895134.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


0
0.005127913784235716
---------
200
0.0037645548582077026
---------
400
0.0029659634456038475
---------
600
0.002141873352229595
---------
800
0.0018871099455282092
---------
1000
0.001799478312022984
---------
1200
0.0017258874140679836
---------
1400
0.0016243611462414265
---------
1600
0.001532913651317358
---------
1800
0.0014811857836320996
---------
2000
0.0014450608287006617
---------
2200
0.0014112046919763088
---------
2400
0.0013770295772701502
---------
2600
0.001336578861810267
---------
2800
0.0012814679648727179
---------
3000
0.001224022009409964
---------
3200
0.0011929180473089218
---------
3400
0.001180856255814433
---------
3600
0.0011756578460335732
---------
3800
0.0011729415273293853
---------
4000
0.0011712646810337901
---------
4200
0.0011700852774083614
---------
4400
0.00116917013656348
---------
4600
0.0011684098280966282
---------
4800
0.0011677492875605822
---------
5000
0.0011671605752781034
---------
5200
0.001166630070656538
---------
5400
0.00116615090

In [40]:
encoder = model.encoder(inputs)
encoder_list = encoder.tolist()
encoder_list

[[1.478389024734497, -0.4531126618385315, 0.0028105974197387695],
 [2.2923765182495117, -0.3491453230381012, 1.3273496627807617],
 [0.9251213073730469, -0.18233001232147217, 1.5837090015411377],
 [0.0009860992431640625, -0.1157941222190857, 0.07974356412887573],
 [1.476144790649414, -0.25407806038856506, -0.16513627767562866],
 [2.2919044494628906, -0.09289124608039856, 1.325872540473938],
 [-2.4199485778808594e-05, -0.5068918466567993, 0.07789519429206848],
 [0.9239853620529175, -0.24356165528297424, 1.5818185806274414],
 [3.2026851177215576, -0.4649634063243866, 1.101290225982666],
 [2.525400161743164, -0.19784456491470337, -0.16330373287200928],
 [0.04213160276412964, -0.09437581896781921, 0.8801637887954712]]

In [46]:
def calc(first, second, third):
    a = [0] * 3
    a[0] = encoder_list[first][0] - encoder_list[second][0] + encoder_list[third][0]
    a[1] = encoder_list[first][1] - encoder_list[second][1] + encoder_list[third][1]
    a[2] = encoder_list[first][2] - encoder_list[second][2] + encoder_list[third][2]
    print(a)

calc(0, 4, 7) # 2
calc(0, 1, 7) # 6
calc(8, 7, 4) # 9

[0.9262295961380005, -0.4425962567329407, 1.7497654557228088]
[0.10999786853790283, -0.34752899408340454, 0.25727951526641846]
[3.754844546318054, -0.4754798114299774, -0.645664632320404]
